In [200]:
using LinearAlgebra;

In [201]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"], "e"=>[])

Dict{String, Array{String, N} where N} with 5 entries:
  "c" => ["a", "d"]
  "e" => []
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [202]:
input=dummy

Dict{String, Array{String, N} where N} with 5 entries:
  "c" => ["a", "d"]
  "e" => []
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [203]:
damping=0.85
function naive_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=zeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        if l>0 
            M[decodeNames[from], decodeNames[from]]=1-damping
            for to in list 
                M[decodeNames[to], decodeNames[from]]=damping/l
            end
        else
            M[decodeNames[from], decodeNames[from]]=1
        end
    end
    return (M, encodeNames, decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [204]:
M, encode, decode=naive_get_matrix(input)

([0.15000000000000002 0.0 … 0.425 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.425 0.0 … 0.15000000000000002 0.0; 0.425 0.0 … 0.0 0.15000000000000002], ["c", "e", "b", "a", "d"], Dict("c" => 1, "e" => 2, "b" => 3, "a" => 4, "d" => 5))

In [205]:
M

5×5 Matrix{Float64}:
 0.15   0.0  0.0   0.425  0.0
 0.0    1.0  0.0   0.0    0.0
 0.0    0.0  0.15  0.425  0.85
 0.425  0.0  0.85  0.15   0.0
 0.425  0.0  0.0   0.0    0.15

In [206]:
function naive(input)
    M, encode, decode = naive_get_matrix(input)
    Λ, X=eigen(M)
    n=size(Λ,1)
    ranks=X[:, n]
    sites=map(r-> (encode[r[1]],Float64(r[2])), enumerate(ranks))
    return sort(sites, by= x->x[2], rev=true)
end

naive (generic function with 1 method)

In [207]:
r=naive(input)

5-element Vector{Tuple{String, Float64}}:
 ("a", 0.7302967433402213)
 ("b", 0.5477225575051661)
 ("c", 0.3651483716701108)
 ("d", 0.1825741858350554)
 ("e", 0.0)